In [2]:
import sys
sys.path.append('../Data')
sys.path.append('../GenerateLabel')


import pandas as pd
import numpy as np
import time
from CommentDataset import CommentDataset
from PageRank import WordGraph, TopicalPageRank

In [3]:
comment_list_name = "20230627_mynumbercard_untenmenkyosyou"
comment_list_path = "2023_06_27/" + comment_list_name + ".csv"

model_path = "model/pickle/2023_06_27/20230627_mynumbercard_untenmenkyosyou_default.pickle"

In [5]:
# 指定すべき文字コードはファイルによる ex.cp932, utf-8
df = pd.read_csv("../Data/yahoo/"+comment_list_path, encoding="utf-8", header=None)
comment_list = df[0][1:].to_numpy().tolist()
comment_dataset = CommentDataset(comment_list)
comment_dataset.delete_comment_specified_string(50)
comment_dataset.formatted_input_hlda(appear_tagging_list=["名詞", "形容詞", "動詞"], stop_tagging_list=["助詞", "助動詞", "非自立"])

In [6]:
from model.HldaModel import ExpandHldaModel

expandHlda = ExpandHldaModel(pickle_path=model_path)

In [18]:
expandHlda.print_nodes(words = 10, weights = True)

topic=0 level=0 (documents=528): 免許 (400), れる (227), 運転 (223), できる (161), 保険 (145), 確認 (132), 思う (121), カード (114), 本人 (111), 情報 (103), 
    topic=1 level=1 (documents=152): 国民 (67), 言う (33), 政府 (32), これ (24), 日本 (21), 岸田 (15), 政治 (14), 自民党 (13), 信用 (12), 支持 (11), 
        topic=2 level=2 (documents=18): 暗証 (12), 番号 (8), 補完 (4), 間違える (4), 違反 (4), 警察官 (4), 読み取り (3), 設定 (3), 大変 (3), 認証 (3), 
        topic=6 level=2 (documents=21): 任意 (13), 信用 (8), 義務 (8), 公安 (5), 実質 (5), 一切 (4), 与える (4), 委員 (3), 姑息 (3), 管轄 (3), 
        topic=10 level=2 (documents=8): 有る (5), 普及 (3), 何処 (2), ゼロ (2), 上がる (2), ども (2), 確約 (2), 無視 (2), 常識 (1), 時期 (1), 
        topic=12 level=2 (documents=11): フォン (6), スマート (6), モバイル (5), 所持 (3), 人間 (2), 悪い (2), 不正 (2), 覗く (1), 壊れる (1), 近く (1), 
        topic=15 level=2 (documents=12): 与党 (12), 潰す (4), 政権 (4), 公明党 (4), 内閣 (4), 存在 (3), 正しい (3), 皆さん (3), 下落 (2), 老人 (2), 
        topic=32 level=2 (documents=7): デジタル (8), 世界 (6), アナログ (5), 人類 (3), 対応 (3), 時点 (2), 進行 (2), 遅れる (2), 

In [15]:
topic_id = 6
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)
tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
topic_word_weighted = expandHlda.get_weighted(topic_id)
phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)

for phrase in phrase_list[0:10]:
    print(phrase)

['任意だから選んだ国民', ['任意', 'だ', 'から', '選ぶ', 'だ', '国民'], 0.07158285288873795]
['実質義務化', ['実質', '義務', '化'], 0.06966310319530553]
['任意のはず', ['任意', 'の', 'はず'], 0.06519140531906019]
['任意', ['任意'], 0.06462175444045536]
['「任意', ['「', '任意'], 0.06462175444045536]
['“任意', ['“', '任意'], 0.06462175444045536]
['顔認証付きカードリーダーを普及するのも苦労しているのにさらに規模を拡大して普及できるもの', ['顔', '認証', '付き', 'カード', 'リーダー', 'を', '普及', 'する', 'の', 'も', '苦労', 'する', 'て', 'いる', 'の', 'に', 'さらに', '規模', 'を', '拡大', 'する', 'て', '普及', 'できる', 'もの'], 0.05415367266344477]
['実質的に取得しなければ不利益を与えるようなやり方', ['実質', '的', 'に', '取得', 'する', 'ない', 'ば', '不利益', 'を', '与える', 'よう', 'だ', 'やり方'], 0.05138424348015635]
['いっそのこと義務', ['いっそ', 'の', 'こと', '義務'], 0.04871571460299793]
['もしれないが「義務', ['も', 'しれる', 'ない', 'が', '「', '義務'], 0.047599303144605296]


In [16]:
topic_id = 24
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)
tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
topic_word_weighted = expandHlda.get_weighted(topic_id)
phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)

for phrase in phrase_list[0:10]:
    print(phrase)

['新しく開設する銀行口座', ['新しい', '開設', 'する', '銀行', '口座'], 0.2703191385369026]
['大切な物を入れてる金庫を無くした もしくは盗まれた実印 銀行口座', ['大切', 'だ', '物', 'を', '入れる', 'てる', '金庫', 'を', '無くす', 'た', 'もしくは', '盗む', 'れる', 'た', '実印', '銀行', '口座'], 0.22814461556008941]
['銀行口座', ['銀行', '口座'], 0.22313567585935928]
['口座開設', ['口座', '開設'], 0.14769625132149788]
['地元都市銀行', ['地元', '都市', '銀行'], 0.144708197939078]
['大手都市銀行４行', ['大手', '都市', '銀行', '４', '行'], 0.1375611622010842]
['しないかは銀行次第', ['する', 'ない', 'か', 'は', '銀行', '次第'], 0.13613895435343354]
['民間銀行', ['民間', '銀行'], 0.12288526884425242]
['どの銀行', ['どの', '銀行'], 0.12061419512524797]
['銀行', ['銀行'], 0.12061419512524797]


In [17]:
topic_id = 484
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)
tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
topic_word_weighted = expandHlda.get_weighted(topic_id)
phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)

for phrase in phrase_list[0:10]:
    print(phrase)

['運転免許更新', ['運転', '免許', '更新'], 0.24399515900266552]
['免許更新', ['免許', '更新'], 0.2399162336262838]
['免許証更新', ['免許', '証', '更新'], 0.2399162336262838]
['ゴールドカードを更新してた人たち', ['ゴールド', 'カード', 'を', '更新', 'する', 'てる', 'た', '人', 'たち'], 0.2316251148326113]
['最低でも更新期間1.2回', ['最低', 'で', 'も', '更新', '期間', '*', '*', '*', '回'], 0.21831695695101466]
['同じような更新手続き', ['同じ', 'よう', 'だ', '更新', '手続き'], 0.20926056299085874]
['更新会場', ['更新', '会場'], 0.20389282016063578]
['更新手続き', ['更新', '手続き'], 0.2020273133055084]
['成人だったら１０年写真を更新しないマイナンバーカード', ['成人', 'だ', 'た', '１', '０', '年', '写真', 'を', '更新', 'する', 'ない', '*'], 0.20056812182993494]
['自動更新', ['自動', '更新'], 0.19663299084968508]


In [19]:
topic_id = 32
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)
tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
topic_word_weighted = expandHlda.get_weighted(topic_id)
phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)

for phrase in phrase_list[0:10]:
    print(phrase)

['アナログ対応', ['アナログ', '対応'], 0.11862763686892508]
['デジタル化', ['デジタル', '化'], 0.1059404006737948]
['必ず起こるアナログ人間', ['必ず', '起こる', 'アナログ', '人間'], 0.08490755304765313]
['世界', ['世界'], 0.07574917517953622]
['アナログ', ['アナログ'], 0.06966498803117288]
['人類史', ['人類', '史'], 0.04225719779771875]
['日常生活', ['日常', '生活'], 0.03800578068314113]
['日本政府', ['日本', '政府'], 0.034019867613059376]
['より良きシステム', ['より', '良い', 'システム'], 0.03100091185966953]
['困難に直面する事', ['困難', 'に', '直面', 'する', '事'], 0.029114331811358897]
